In [1]:
import os
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from PIL import Image
Image.MAX_IMAGE_PIXELS = None
import pyvips
import cv2

C:\Users\Marina\anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
data = pd.read_csv(r'D:\4th year\Deep Learning\project\train.csv')
image_dir = r'D:\4th year\Deep Learning\project\train_thumbnails'
masks_dir = r'D:\4th year\Deep Learning\project\masks'
masked_images_dir = r'D:\4th year\Deep Learning\masked_images'
output_dir = r"D:\4th year\Deep Learning\project\masked_images"  # Directory to save the masked images


In [3]:
label_mapping = {
    "CC": 0,
    "EC": 1,
    "HGSC": 2,
    "LGSC": 3,
    "MC": 4,
}

In [4]:
preprocessed_data = []

for filename in os.listdir(masked_images_dir):
        
    # extract image id
    img_path = os.path.join(masked_images_dir, filename)
    image_id = filename.split('_')[1].split('.')[0]
    img = Image.open(img_path)
    masked_img = np.array(img)
        
    mask_path = os.path.join(masks_dir, f"{image_id}.png")
    mask = pyvips.Image.new_from_file(mask_path)
    # Calculate the scaling factors for resizing the mask
    scale_x = img.width / mask.width
    scale_y = img.height / mask.height
    # Resize the mask to fit the dimensions of the image
    mask = mask.resize(scale_x, vscale=scale_y)
    
    # Find the coordinates of non-zero pixels in the mask
    non_zero_pixels = np.transpose(np.nonzero(mask))

    if non_zero_pixels.size != 0:
        # Get the minimum and maximum coordinates along each axis to form the bounding box
        x_min = np.min(non_zero_pixels[:, 1])
        y_min = np.min(non_zero_pixels[:, 0])
        x_max = np.max(non_zero_pixels[:, 1])
        y_max = np.max(non_zero_pixels[:, 0])

        # Calculate the bounding box dimensions
        x, y, w, h = x_min, y_min, x_max - x_min, y_max - y_min

        if w > 0 and h > 0:  # Ensure valid dimensions
            # Crop the region from the original image using the bounding box
            cropped_region = masked_img[y:y+h, x:x+w]

#             # Display the cropped region
#             plt.imshow(cropped_region, cmap='gray')
#             plt.title(f'Cropped Region {image_id}')
#             plt.show()

            # extract label
            label = data[data['image_id'] == int(image_id)]['label'].values  
            label = label[0]  
            numerical_label = label_mapping[label]
            one_hot_label = np.zeros(len(label_mapping))
            one_hot_label[numerical_label] = 1

            preprocessed_data.append((cropped_region, one_hot_label))
            
            cropped_image_filename = f"cropped_{image_id}.png"
            cropped_image_path = os.path.join(output_dir, cropped_image_filename)

            # Convert the resulting numpy array back to a pyvips image
            result_pyvips_image = pyvips.Image.new_from_memory(cropped_region.data.tobytes(), cropped_region.shape[1], cropped_region.shape[0], cropped_region.shape[2], "uchar")

            # Save the resulting image with a unique name
            result_pyvips_image.write_to_file(cropped_image_path)

        else:
            print(f"Invalid dimensions for cropping: {image_id}")
    else:
        print(f"No non-zero pixels found in the mask: {image_id}")



In [5]:
from sklearn.model_selection import train_test_split

train_data, val_data = train_test_split(preprocessed_data, test_size=0.2, random_state=42)


# Extract features and labels from the train_data and val_data
train_images, train_labels = zip(*train_data)
val_images, val_labels = zip(*val_data)


# Resize images to a common size
train_images= [cv2.resize(img, (512,512)) for img in train_images]
val_images = [cv2.resize(img,  (512,512)) for img in val_images]


# Convert lists to numpy arrays
train_images = np.array(train_images)
train_labels = np.array(train_labels)
val_images = np.array(val_images)
val_labels = np.array(val_labels)

In [6]:
num_classes = 5
batch_size = 8

# the model
base_model = tf.keras.applications.ResNet50(
    include_top=False, weights='imagenet', input_shape=(512, 512, 3)
)

flatten = tf.keras.layers.Flatten()
fc1 = tf.keras.layers.Dense(num_classes, activation='softmax')

inputs = tf.keras.Input(shape=(512, 512, 3))
x = base_model(inputs)
x = flatten(x)
outputs = fc1(x)


model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# model.fit(
#     train_images, train_labels,
#     batch_size=batch_size,
#     epochs=20,
#     validation_data=(val_images, val_labels)
# )

Epoch 1/20
16/16 [==============================] - 121s 7s/step - loss: 9.9285 - accuracy: 0.2645 - val_loss: 5.0118 - val_accuracy: 0.2581
Epoch 2/20
16/16 [==============================] - 105s 7s/step - loss: 1.1170 - accuracy: 0.8512 - val_loss: 9.3089 - val_accuracy: 0.2258
Epoch 3/20
16/16 [==============================] - 106s 7s/step - loss: 0.2298 - accuracy: 0.9669 - val_loss: 5.7630 - val_accuracy: 0.3226
Epoch 4/20
 5/16 [========>.....................] - ETA: 1:17 - loss: 1.3709e-07 - accuracy: 1.0000

KeyboardInterrupt: 

In [7]:
from sklearn.metrics import balanced_accuracy_score


train_balanced_acc = []
val_balanced_acc = []

for epoch in range(20):  # Assuming 20 epochs
    print(f"Epoch {epoch + 1}/{20}")
    
    # Training step
    model.fit(
        train_images, train_labels,
        batch_size=batch_size,
        epochs=1,  # Train for 1 epoch per iteration
        validation_data=(val_images, val_labels)
    )

    # Predictions on training and validation data
    train_pred = model.predict(train_images)
    val_pred = model.predict(val_images)

    # Calculate balanced accuracy for training and validation
    train_acc = balanced_accuracy_score(np.argmax(train_labels, axis=1), np.argmax(train_pred, axis=1))
    val_acc = balanced_accuracy_score(np.argmax(val_labels, axis=1), np.argmax(val_pred, axis=1))

    print(f"Train balanced accuracy: {train_acc}")
    print(f"Validation balanced accuracy: {val_acc}")

    # Store metrics for visualization or further analysis
    train_balanced_acc.append(train_acc)
    val_balanced_acc.append(val_acc)


Epoch 1/20
1/1 [==============================] - 5s 5s/step
Train balanced accuracy: 0.9800000000000001
Validation balanced accuracy: 0.1898989898989899
Epoch 2/20
1/1 [==============================] - 5s 5s/step
Train balanced accuracy: 0.9666666666666668
Validation balanced accuracy: 0.29494949494949496
Epoch 3/20
 2/16 [==>...........................] - ETA: 1:34 - loss: 0.1942 - accuracy: 0.9375    

KeyboardInterrupt: 